# imports

In [8]:
#----------------------------------------------
#          IMPORT PYTHON LIBRARIES
#----------------------------------------------
from matplotlib.pyplot import figure
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.spatial
import numpy as np
import urllib.request as urllib2
from sklearn.feature_extraction.text import CountVectorizer
from skimage import io
import difflib
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import copy 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.metrics import accuracy_score
from scipy.spatial import distance
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from scipy.stats import pearsonr
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error

# **Google Drive Access**

---




In [ ]:
#----------------------------------------------
#             MOUNT GOOGLE DRIVE    
#----------------------------------------------  
from google.colab import drive
drive.mount('/content/drive')
#----------------------------------------------
#           PRINT FILES IN DIRECTORY 
#----------------------------------------------
!ls "/content/drive/My Drive"
!pwd

# **Functions**

---





In [9]:
def novelty(user_id,items):
  products = data.loc[data['UserId'] == user_id]
  products = products.set_index('ProductNo')
  rec_items = products.loc[items]['clicks'].tolist()
  return round ( (len(rec_items) - len(np.array(np.nonzero(rec_items)).squeeze())) / len(rec_items),2)


def coverage(total_rec,already_rated):
  
  return round( (len(total_rec) / (4325-already_rated)),3)

def diversity(items):
  matrix = cosine_similarity(np.array(cloth_attr.loc[items]))
  return round(1-np.mean(matrix[np.nonzero(np.triu(matrix,1))]),3)
import re
def lower_dimension_frame2(dataset,attr):
  # initialize category array
  categories = []
  for att in attr:
    # get all unique categories, split them from comma
    for i in dataset[att].unique():
      if type(i) == float:
        i = 'NAN'
      splited_ = re.split('/|,|to|and| |&|-|',i)
      splited_ = [x.lower() for x in splited_]
      for j in range(len(splited_)):
        if splited_[j] not in categories:
          categories.append(splited_[j])

  # create a new array that contains all those categories
  result_frame = pd.DataFrame()
  result_array = np.zeros((len(dataset[attr]),len(categories)))
  for att in attr:
    for i in range(len(dataset[att])):
      if type(dataset[att].tolist()[i]) == float:
        splited_ = 'NAN'.split('/')
      else:
        splited_ = re.split('/|,|to|and| |&|-|',dataset[att].tolist()[i])
        splited_ = [x.lower() for x in splited_]
      for j in range(len(splited_)):
        for k in range(len(categories)):
          if splited_[j] == categories[k]:
            result_array[i][k] = 1

  result_frame = pd.DataFrame(result_array,
                    columns= categories)


  index_frame = dataset['ProductNo'].tolist()
  result_frame['index'] = index_frame
  result_frame = result_frame.set_index('index')
  return result_frame
def lower_dimension_frame(dataset,attr):
  # initialize category array
  categories = []
  for att in attr:
    # get all unique categories, split them from comma
    for i in dataset[att].unique():
      if type(i) == float:
        i = 'NAN'
      splited_ = i.split(',')
      for j in range(len(splited_)):
        if splited_[j] not in categories:
          categories.append(splited_[j])

  # create a new array that contains all those categories
  result_frame = pd.DataFrame()
  result_array = np.zeros((len(dataset[attr]),len(categories)))
  for att in attr:
    for i in range(len(dataset[att])):
      if type(dataset[att].tolist()[i]) == float:
        splited_ = 'NAN'.split(',')
      else:
        splited_ = dataset[att].tolist()[i].split(',')
      for j in range(len(splited_)):
        for k in range(len(categories)):
          if splited_[j] == categories[k]:
            result_array[i][k] = 1

  result_frame = pd.DataFrame(result_array,
                    columns= categories)


  index_frame = dataset['ProductNo'].tolist()
  result_frame['index'] = index_frame
  result_frame = result_frame.set_index('index')
  return result_frame

def user_input_dataset():
  # load dataset to temporary variable
  temp_dataset = data

  # available search words in dataset
  all_searchWords = data['SearchWords'].tolist()

  # make search words to lower case
  all_searchWords = [x.lower() for x in all_searchWords]

  # remove duplicate search words
  all_searchWords = set(all_searchWords)

  # info about search words
  print('There are total: ',len(all_searchWords),' search words ')

  # user searching clothes category
  input_searchWords = input("Please enter your search words:  ")

  # user searching clothes category
  numOf_relatedWords = input("Please enter the number of releated words we wanna search :  ")
  numOf_relatedWords = int(numOf_relatedWords)

  # find matches based on dataset searchwords
  matches = difflib.get_close_matches(input_searchWords, all_searchWords, numOf_relatedWords )

  # print matches
  print('Result found: ',matches)

  # merge result of matches
  dataFrames = []
  for i in matches:
    dataFrames.append(data.loc[data['SearchWords'] == i.upper()])

  # all available clothes based on user search
  totalClothes = pd.concat(dataFrames)

  temp_dataset = totalClothes

  return temp_dataset

def frame_merger(dataset):
  merged_frame = pd.merge(prod_frame,fit_frame,on='index')
  #merged_frame = pd.merge(merged_frame,length_frame,on='index')
  #merged_frame = pd.merge(merged_frame,color_frame,on='index')
  merged_frame = pd.merge(merged_frame,neck_frame,on='index')
  merged_frame = pd.merge(merged_frame,sleeve_frame,on='index')
  merged_frame = pd.merge(merged_frame,search_word_Dataframe,on='index')
  merged_frame['url'] = dataset['ImageSource'].tolist()
  merged_frame['gradeSystem'] = dataset['gradeSystem'].tolist()
  merged_frame['grade'] = dataset['gradeUser'].tolist()
  
  #merged_frame['dashboard'] = dataset['dashboard'].tolist()
  #merged_frame['favorite'] = dataset['favorite'].tolist()
  #merged_frame['clicks'] = dataset['clicks'].tolist()
  url_all = merged_frame['url']
  grades_all = merged_frame['grade'].tolist()
  grade_sys = merged_frame['gradeSystem'].tolist()
  merged_frame = merged_frame.drop(['grade'],axis=1)
  merged_frame = merged_frame.drop(['gradeSystem'],axis=1)
  merged_frame = merged_frame.drop(['url'],axis=1)
  return merged_frame,grades_all,grade_sys,url_all


def recommend_clothes(model):
    # ask query from the user
  feed = user_input_dataset()

  # get all clothes from query (rated and unrated)
  queryFeed = merged_frame.loc[ feed['ProductNo'].tolist() ]

  # get rated clothes from query
  ratedClothes = queryFeed.loc[queryFeed['gradeUser'] !=5]

  # get rated clothes from query
  unratedClothes = queryFeed.loc[queryFeed['gradeUser'] ==5]

  # keep grade system
  gradeSys = unratedClothes['gradeSystem']

  # keep image source
  url = unratedClothes['url']

  # delete grade user == 5
  unratedClothes = unratedClothes.drop(labels='gradeUser',axis=1)

  # delete grade system
  unratedClothes = unratedClothes.drop(labels='gradeSystem',axis=1)

  # delete image source
  unratedClothes = unratedClothes.drop(labels='url',axis=1)

  # predict ratings
  pred = model.predict(unratedClothes)

  # update dataframe with predicted ratings
  unratedClothes['predUser'] = pred

  # update dataframe with grade system
  unratedClothes['gradeSystem'] = gradeSys

  # update dataframe with image source
  unratedClothes['url'] = url

  # get well rated clothes for recommendations
  unratedClothes = unratedClothes.loc[unratedClothes['predUser'] == 1]

  # sort clothes based on grade system
  unratedClothes = unratedClothes.sort_values(by='gradeSystem',ascending=True)
  
  return ratedClothes,unratedClothes


def show_recommended_clothes(unratedClothes):
  # print recommendations
  for i in range(len(unratedClothes)):
    cloth = unratedClothes.iloc[i]
    rec_cloth = cloth['url']
    print('-----------------')  
    image = io.imread(rec_cloth)
    plt.imshow(image)
    plt.show()


def show_liked_clothes(ratedClothes):
  # print liked clothes
  likedClothes = ratedClothes.loc[ratedClothes['gradeUser']>=8]
  for i in range(len(likedClothes)):
    cloth = likedClothes.iloc[i]
    rec_cloth = cloth['url']
    print('-----------------')  
    image = io.imread(rec_cloth)
    plt.imshow(image)
    plt.show()
  
def query_accuracy(ratedClothes,model):
  gradeUser = [1 if x>=8 else 0 for x in ratedClothes['gradeUser'].tolist()]
  pred_frame = ratedClothes
  pred_frame = pred_frame.drop(['gradeUser','gradeSystem','url'],axis=1)
  predUser = model.predict(pred_frame)
  return accuracy_score(gradeUser,predUser)


# load multi data

In [4]:
import pandas as pd
# read dataset 
all_data = pd.read_csv('/content/drive/My Drive/final.csv')
all_data
data = all_data.loc[all_data['UserId'] == 0]
data

,Unnamed: 0,UserId,ProductNo,Crawler,SearchWords,Image,url,ImageSource,SiteClothesHeadline,Color,Gender,Brand,Metadata,ProductCategory,ProductSubcategory,Length,Sleeve,CollarDesign,NeckDesign,Fit,gradeSystem,clicks,gradeUser,favorite,dashboard
0,0,0,0,Asos,SHORTSET,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/in-the-style/in-the-style...,https://images.asos-media.com/products/in-the-...,In The Style x Billie Faiers motif t shirt and...,White,WOMAN,In The Style,Collaboration with influencer Billie Faiers - ...,SET,"SHIRT,T-SHIRT",SHORT LENGTH,SHORT SLEEVE,NO COLLAR,ROUND NECK,RELAXED FIT,0,1,5,0,1
1,1,0,1,Asos,SHORTSET,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/bluebella/bluebella-abiga...,https://images.asos-media.com/products/bluebel...,Bluebella Abigail Satin Short Pyjama Set,Pink,WOMAN,BlueBella,You’re invited to this PJ party - Notch lapels...,SET,SET,SHORT LENGTH,SHORT SLEEVE,POLO COLLAR,COLLAR,REGULAR FIT,1,1,8,1,0
2,2,0,2,Asos,SHORTSET,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/chelsea-peers/chelsea-pee...,https://images.asos-media.com/products/chelsea...,Chelsea Peers stripe flamingo printed cami sho...,Multi,WOMAN,Chelsea Peers,Tonight's agenda: early night - V-neck - Adjus...,SET,SET,SHORT LENGTH,SLEEVELESS,NO COLLAR,V NECK,REGULAR FIT,2,1,5,0,0
3,3,0,3,Asos,SHORTSET,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/lindex/lindex-sou-jenna-s...,https://images.asos-media.com/products/lindex-...,Lindex SoU Jenna spot printed organic cotton t...,Black and white,WOMAN,Lindex,Worn and shot by one of our models at home - P...,SET,"T-SHIRT,SHORT SET",SHORT LENGTH,SHORT SLEEVE,MAO COLLAR,COLLAR,REGULAR FIT,3,1,6,0,0
4,4,0,4,Asos,SHORTSET,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/chelsea-peers/chelsea-pee...,https://images.asos-media.com/products/chelsea...,Chelsea Peers abstract leaf print revere pyjam...,Multi,WOMAN,Chelsea Peers,Tonight's agenda: early night - Pyjama set - R...,SET,"SHORT SET,T-SHIRT",SHORT LENGTH,SLEEVELESS,NO COLLAR,COLLAR,REGULAR FIT,4,0,9,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688,0,8688,Asos,HALTERNECKPYJAMAS,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/asos-design/asos-design-l...,https://images.asos-media.com/products/asos-de...,ASOS DESIGN lounge pyjama shorts in black with...,Black,WOMAN,ASOS DESIGN,Part of our responsible edit - Branded waistba...,NaN,SHORTS,KNEE LENGTH,LONG SLEEVE,NO COLLAR,OFF SHOULDER,REGULAR FIT,0,0,5,0,0
8689,8689,0,8689,Asos,HALTERNECKPYJAMAS,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/asos-design/asos-design-f...,https://images.asos-media.com/products/asos-de...,ASOS DESIGN floral shirt & trouser pyjama set ...,Black floral,WOMAN,ASOS DESIGN,Part of our responsible edit - Floral print - ...,SET,SHIRT,MEDIUM LENGTH,FLARED,MAO COLLAR,COLLAR,REGULAR FIT,0,0,5,0,0
8690,8690,0,8690,Asos,HALTERNECKPYJAMAS,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/asos-design/asos-design-m...,https://images.asos-media.com/products/asos-de...,ASOS DESIGN mix & match jersey pyjama cropped ...,Mint,WOMAN,ASOS DESIGN,Entering sleep mode - V-neck - Three-quarter-l...,NaN,TOP,SHORT LENGTH,LONG SLEEVE,NO COLLAR,V NECK,SLIM FIT,0,0,5,0,0
8691,8691,0,8691,Asos,HALTERNECKPYJAMAS,/home/alexandros/Desktop/createCsvDiplo/AsosIm...,https://www.asos.com/topshop/topshop-satin-pyj...,https://images.asos-media.com/products/topshop...,Topshop satin pyjama set in apple green,Green,WOMAN,Topshop,Tonight's agenda: early night - Pyjama set - S...,SET,"SET,TOP",SHORT LENGTH,SHORT SLEEVE,POLO COLLAR,COLLAR,REGULAR FIT,0,0,5,0,0


# preprocessing data

## **Preprocessing Fit**

Reduced categories from 16 to 5

In [10]:
su = all_data.loc[all_data['UserId'] == 0]
fit_frame = lower_dimension_frame(su,['Fit'])

In [11]:
len(all_data['Fit'].unique())

16

In [12]:
len(fit_frame.columns)

5

In [13]:
fit_frame

,RELAXED FIT,REGULAR FIT,SLIM FIT,CARGO,BIKER
index,,,,,
0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...
8688,0.0,1.0,0.0,0.0,0.0
8689,0.0,1.0,0.0,0.0,0.0
8690,0.0,0.0,1.0,0.0,0.0


## **Preprocessing Neck Design**

Reduced categories from 20 to 7

In [14]:
su = all_data.loc[all_data['UserId'] == 0]
neck_frame = lower_dimension_frame(su,['NeckDesign'])
neck_frame

,ROUND NECK,COLLAR,V NECK,HOODED,OFF SHOULDER,HALTERNECK,TURTLENECK
index,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
8688,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [15]:
all_data['NeckDesign'].unique()

array(['ROUND NECK', 'COLLAR', 'V NECK', 'HOODED', 'OFF SHOULDER',
       'HALTERNECK', 'TURTLENECK', 'COLLAR,V NECK', 'COLLAR,HOODED',
       'TURTLENECK,COLLAR', 'ROUND NECK,V NECK', 'V NECK,COLLAR',
       'COLLAR,OFF SHOULDER', 'OFF SHOULDER,V NECK',
       'OFF SHOULDER,COLLAR', 'COLLAR,ROUND NECK', 'V NECK,OFF SHOULDER',
       'HOODED,COLLAR', 'HALTERNECK,V NECK', 'HALTERNECK,ROUND NECK'],
      dtype=object)

In [16]:
len(all_data['NeckDesign'].unique())

20

In [17]:
len(neck_frame.columns)

7

## **Preprocessing Length**

Reduced categories from 7 to 6

In [18]:
su = all_data.loc[all_data['UserId'] == 0]
length_frame = lower_dimension_frame(su,['Length'])
length_frame

,SHORT LENGTH,MEDIUM LENGTH,KNEE LENGTH,LONG,CAPRI,3/4 LENGTH
index,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
8688,0.0,0.0,1.0,0.0,0.0,0.0
8689,0.0,1.0,0.0,0.0,0.0,0.0
8690,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
len(all_data['Length'].unique())

7

In [20]:
len(length_frame.columns)

6

## **Preprocessing Product Category & SubCategory**

Reduced categories from 179 to 33

In [21]:
su = all_data.loc[all_data['UserId'] == 0]
# 'ProductCategory','ProductSubcategory'
prod_frame = lower_dimension_frame(su,['ProductCategory','ProductSubcategory'])
prod_frame


,SET,PYJAMAS,SKIRT,NAN,DRESS,TROUSERS,LEGGINGS,TRACKSUIT,CARDIGAN,ROMPER,SHIRTS,SHIRT,T-SHIRT,SHORT SET,TOP,JOGGERS,SKIRT SET,JACKET,SHORTS,BLOUSE,VEST,JEANS,SWEATSHIRT,BIKINI,BERMUDA SHORTS,JUMPSUIT,DUNGAREES,PLAYSUIT,SLEEVELESS BLOUSE,BOMBER JACKET,BLAZER,TUNIC,SWIMSUIT
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8689,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
len(data['ProductSubcategory'].unique())+ len(data['ProductCategory'].unique())

179

In [23]:
len(prod_frame.columns)

33

## **Preprocessing Search-Word**

In [24]:
su = all_data.loc[all_data['UserId'] == 0]
single_user_data = su.copy()
unique_search_words = single_user_data['SearchWords'].unique()
future_frame = []
for i in single_user_data['ImageSource'].tolist():
  z_array = np.zeros((1,len(unique_search_words)))
  local_sw = su.loc[su['ImageSource'] == i]['SearchWords'].tolist()
  #print(local_sw)
  for j in range(len(unique_search_words)):
    for k in range(len(local_sw)):
      if local_sw[k] == unique_search_words[j]:
        #print(local_sw[k])
        z_array[0][j] = 1
  future_frame.append(z_array)


In [25]:
future_frame = np.array(future_frame).squeeze()
search_word_Dataframe = pd.DataFrame(data=future_frame,columns=unique_search_words)
index = prod_frame.index.tolist()
search_word_Dataframe['index'] = index
search_word_Dataframe = search_word_Dataframe.set_index(keys='index')
np.array(search_word_Dataframe.columns.tolist())

array(['SHORTSET', 'LONGSET', 'MEDIUMSET', 'KNEESET', 'CAPRISET',
       '3/4LENGTHSET', 'SHORTSLEEVESET', 'LONGSLEEVESET',
       'TURNUPSLEEVESET', 'SLEEVELESSSET', 'RAGLANSLEEVESET',
       'CUPSLEEVESET', '3/4FLAREDSET', 'FLAREDSET', 'POLOCOLLARSET',
       'SHIRTCOLLARSET', 'FLATKNITTEDRIBSET', 'MAOCOLLARSET',
       'STANDUPCOLLARSET', 'ROUNDNECKSET', 'COLLARSET', 'TURTLENECKSET',
       'HOODEDSET', 'VNECKSET', 'OFFSHOULDERSET', 'HALTERNECKSET',
       'SHORTBERMUDAS-SHORTS', 'LONGBERMUDAS-SHORTS',
       'MEDIUMBERMUDAS-SHORTS', 'KNEEBERMUDAS-SHORTS',
       'CAPRIBERMUDAS-SHORTS', '3/4LENGTHBERMUDAS-SHORTS',
       'SHORTSLEEVEBERMUDAS-SHORTS', 'LONGSLEEVEBERMUDAS-SHORTS',
       'TURNUPSLEEVEBERMUDAS-SHORTS', 'SLEEVELESSBERMUDAS-SHORTS',
       'RAGLANSLEEVEBERMUDAS-SHORTS', 'CUPSLEEVEBERMUDAS-SHORTS',
       '3/4FLAREDBERMUDAS-SHORTS', 'FLAREDBERMUDAS-SHORTS',
       'POLOCOLLARBERMUDAS-SHORTS', 'SHIRTCOLLARBERMUDAS-SHORTS',
       'FLATKNITTEDRIBBERMUDAS-SHORTS', 'MAOCOLLAR

In [26]:
search_word_Dataframe.columns.tolist()

['SHORTSET',
 'LONGSET',
 'MEDIUMSET',
 'KNEESET',
 'CAPRISET',
 '3/4LENGTHSET',
 'SHORTSLEEVESET',
 'LONGSLEEVESET',
 'TURNUPSLEEVESET',
 'SLEEVELESSSET',
 'RAGLANSLEEVESET',
 'CUPSLEEVESET',
 '3/4FLAREDSET',
 'FLAREDSET',
 'POLOCOLLARSET',
 'SHIRTCOLLARSET',
 'FLATKNITTEDRIBSET',
 'MAOCOLLARSET',
 'STANDUPCOLLARSET',
 'ROUNDNECKSET',
 'COLLARSET',
 'TURTLENECKSET',
 'HOODEDSET',
 'VNECKSET',
 'OFFSHOULDERSET',
 'HALTERNECKSET',
 'SHORTBERMUDAS-SHORTS',
 'LONGBERMUDAS-SHORTS',
 'MEDIUMBERMUDAS-SHORTS',
 'KNEEBERMUDAS-SHORTS',
 'CAPRIBERMUDAS-SHORTS',
 '3/4LENGTHBERMUDAS-SHORTS',
 'SHORTSLEEVEBERMUDAS-SHORTS',
 'LONGSLEEVEBERMUDAS-SHORTS',
 'TURNUPSLEEVEBERMUDAS-SHORTS',
 'SLEEVELESSBERMUDAS-SHORTS',
 'RAGLANSLEEVEBERMUDAS-SHORTS',
 'CUPSLEEVEBERMUDAS-SHORTS',
 '3/4FLAREDBERMUDAS-SHORTS',
 'FLAREDBERMUDAS-SHORTS',
 'POLOCOLLARBERMUDAS-SHORTS',
 'SHIRTCOLLARBERMUDAS-SHORTS',
 'FLATKNITTEDRIBBERMUDAS-SHORTS',
 'MAOCOLLARBERMUDAS-SHORTS',
 'STANDUPCOLLARBERMUDAS-SHORTS',
 'ROUNDNECKBERMUD

In [27]:
len(search_word_Dataframe)

8693

# merge

In [28]:
merged = pd.merge(prod_frame,length_frame,on='index')
merged = pd.merge(merged,search_word_Dataframe,on='index')
merged = pd.merge(merged,fit_frame,on='index')
merged = pd.merge(merged,neck_frame,on='index')
merged

,SET,PYJAMAS,SKIRT,NAN,DRESS,TROUSERS,LEGGINGS,TRACKSUIT,CARDIGAN,ROMPER,SHIRTS,SHIRT,T-SHIRT,SHORT SET,TOP,JOGGERS,SKIRT SET,JACKET,SHORTS,BLOUSE,VEST,JEANS,SWEATSHIRT,BIKINI,BERMUDA SHORTS,JUMPSUIT,DUNGAREES,PLAYSUIT,SLEEVELESS BLOUSE,BOMBER JACKET,BLAZER,TUNIC,SWIMSUIT,SHORT LENGTH,MEDIUM LENGTH,KNEE LENGTH,LONG,CAPRI,3/4 LENGTH,SHORTSET,...,OFFSHOULDERSWIMMINGSUITS,HALTERNECKSWIMMINGSUITS,SHORTPYJAMAS,LONGPYJAMAS,MEDIUMPYJAMAS,KNEEPYJAMAS,CAPRIPYJAMAS,3/4LENGTHPYJAMAS,SHORTSLEEVEPYJAMAS,LONGSLEEVEPYJAMAS,TURNUPSLEEVEPYJAMAS,SLEEVELESSPYJAMAS,RAGLANSLEEVEPYJAMAS,CUPSLEEVEPYJAMAS,3/4FLAREDPYJAMAS,FLAREDPYJAMAS,POLOCOLLARPYJAMAS,SHIRTCOLLARPYJAMAS,FLATKNITTEDRIBPYJAMAS,MAOCOLLARPYJAMAS,STANDUPCOLLARPYJAMAS,ROUNDNECKPYJAMAS,COLLARPYJAMAS,TURTLENECKPYJAMAS,HOODEDPYJAMAS,VNECKPYJAMAS,OFFSHOULDERPYJAMAS,HALTERNECKPYJAMAS,RELAXED FIT,REGULAR FIT,SLIM FIT,CARGO,BIKER,ROUND NECK,COLLAR,V NECK,HOODED,OFF SHOULDER,HALTERNECK,TURTLENECK
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8690,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [29]:
dataset = data.drop_duplicates(subset='ImageSource',keep='first')
#dataset = dataset.loc[dataset['gradeUser'] !=2.5]
grades = dataset['gradeUser'].tolist()
index = dataset['ProductNo'].tolist()
url = np.array(dataset['ImageSource'].tolist())
merged = merged.loc[index]
merged = merged.reset_index()
merged = merged.drop(columns='index')
merged

,SET,PYJAMAS,SKIRT,NAN,DRESS,TROUSERS,LEGGINGS,TRACKSUIT,CARDIGAN,ROMPER,SHIRTS,SHIRT,T-SHIRT,SHORT SET,TOP,JOGGERS,SKIRT SET,JACKET,SHORTS,BLOUSE,VEST,JEANS,SWEATSHIRT,BIKINI,BERMUDA SHORTS,JUMPSUIT,DUNGAREES,PLAYSUIT,SLEEVELESS BLOUSE,BOMBER JACKET,BLAZER,TUNIC,SWIMSUIT,SHORT LENGTH,MEDIUM LENGTH,KNEE LENGTH,LONG,CAPRI,3/4 LENGTH,SHORTSET,...,OFFSHOULDERSWIMMINGSUITS,HALTERNECKSWIMMINGSUITS,SHORTPYJAMAS,LONGPYJAMAS,MEDIUMPYJAMAS,KNEEPYJAMAS,CAPRIPYJAMAS,3/4LENGTHPYJAMAS,SHORTSLEEVEPYJAMAS,LONGSLEEVEPYJAMAS,TURNUPSLEEVEPYJAMAS,SLEEVELESSPYJAMAS,RAGLANSLEEVEPYJAMAS,CUPSLEEVEPYJAMAS,3/4FLAREDPYJAMAS,FLAREDPYJAMAS,POLOCOLLARPYJAMAS,SHIRTCOLLARPYJAMAS,FLATKNITTEDRIBPYJAMAS,MAOCOLLARPYJAMAS,STANDUPCOLLARPYJAMAS,ROUNDNECKPYJAMAS,COLLARPYJAMAS,TURTLENECKPYJAMAS,HOODEDPYJAMAS,VNECKPYJAMAS,OFFSHOULDERPYJAMAS,HALTERNECKPYJAMAS,RELAXED FIT,REGULAR FIT,SLIM FIT,CARGO,BIKER,ROUND NECK,COLLAR,V NECK,HOODED,OFF SHOULDER,HALTERNECK,TURTLENECK
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4320,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4321,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4322,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4323,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,